## Workings imports

In [ ]:
import sys
import os
sys.path.insert(0, os.getcwd()+'/src/')
sys.path.insert(0, '/home/mcarenso/code/stage-2023-multiscale-extremes/scripts/src/')
from myImports import *
%matplotlib inline
%load_ext autoreload

## Filenames and global variables

In [ ]:
stringSST = "300" ##295, 300 or 305
n_days = 25
chunk_size = {'time' :1, 'x' : 2048, 'y' : 128}
label_chunk_size = {'time' :1, 'longitude' : 2048, 'latitude' : 128}

#TOOCAN segmentation masks (les labels des objets MCS, dans la grille originale x,y,t)
file_seg='/bdd/MT_WORKSPACE/MCS/RCE/SAM/TOOCAN/TOOCAN_v2022_04/irtb/TOOCAN_2.07_SAM_RCE_large'+stringSST+'_2D_irtb.nc'

# TOOCAN objects (list d'objets MCS, leur labels et leur caractéristiques)
file_tracking='/bdd/MT_WORKSPACE/MCS/RCE/SAM/TOOCAN/TOOCAN_v2022_04/irtb/FileTracking/TOOCAN-SAM_RCE_large'+stringSST+'_2D_irtb.dat.gz'

sam_dir_path = "/bdd/MT_WORKSPACE/REMY/RCEMIP/SAM/300K/"

: 

## Precip dataset

In [6]:
# Open native precip datasets
ds1 = xr.open_dataset(sam_dir_path+"rcemip_large_2048x128x74_3km_12s_"+stringSST+"K_64.2Dcom_1.nc")
ds2 = xr.open_dataset(sam_dir_path+"rcemip_large_2048x128x74_3km_12s_"+stringSST+"K_64.2Dcom_2.nc")

# Combine datasets
ds = xr.concat([ds1, ds2], dim='time')
# Rename dimensions
ds["x"] = ((ds["x"])/3e3).astype(int)
ds["y"] = (ds["y"]/3e3).astype(int) 
ds["time"] = np.round(((ds["time"]-75)*48)).astype(int)
ds = ds.isel(time = slice(0, 48*n_days)) 

distributed.diskutils - INFO - Found stale lock file and directory '/home/mcarenso/dask-worker-space/worker-c7xmvkck', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/mcarenso/dask-worker-space/worker-0b41cf_h', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/mcarenso/dask-worker-space/worker-0pccjfhk', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/mcarenso/dask-worker-space/worker-gz_lumiy', purging


## Import or Compute Precip Distrib

In [4]:
filename = 'dist_SAM_Native_Prec_sizemax_1e7.pkl' # used for import or saving the object
path = '/homedata/mcarenso/distrib/'  # my desired directory path

# Check if the file exists
if os.path.isfile(os.path.join(path, filename)):
    # File exists, load the object
    with open(os.path.join(path, filename), 'rb') as file:
        dist_SAM_Prec = pickle.load(file)
else:
    # File doesn't exist, create the object

    dist_SAM_Prec = cs.Distribution(name="SAM Precipitation", bintype = "invlogQ", nd = 6, fill_last_decade=True)
    dist_SAM_Prec.computeDistribution(sample = ds["Prec"].stack(flat=('time', 'y', 'x')).to_numpy())
    dist_SAM_Prec.storeSamplePoints(sample = ds["Prec"].stack(flat=('time', 'y', 'x')).to_numpy(), sizemax = int(1e7))

    # Save the object as a file
    with open(os.path.join(path, filename), 'wb') as file:
        pickle.dump(dist_SAM_Prec, file)
        
size = dist_SAM_Prec.size
ds_SAM_shape = (size//(128*2048), 128, 2048)

NameError: name 'ds' is not defined

## Import MCS data. List and Labels over (t,y,x)

In [7]:
## Import MCS list and prepare label list

from load_TOOCAN_DYAMOND_modif_BenAndMax import load_TOOCAN_DYAMOND
MCS = load_TOOCAN_DYAMOND(file_tracking)

label_list = [MCS[i].label for i in range(len(MCS))]

## function to retrieve the indexes in MCS by MCS labels, could be put in myFuncs but need label_list from the tracking file

def idx_by_label(labels):
    idxs = [label_list.index(label) for label in labels]
    return idxs

In [8]:
label_mask = xr.open_dataarray(file_seg).isel(time=slice(48*n_days)).chunk(label_chunk_size)

## function to retrieve the indexes in MCS by MCS labels, could be put in myFuncs but need label_list from the tracking file
MCS_6h = [MCS[i] for i in range(len(MCS)) if MCS[i].duration in [12]] ## TODO : add the symetry if here
MCS_6h_labels = [MCS_6h[i].label for i in range(len(MCS_6h))]

def idx_by_label(labels, label_list = label_list):
    idxs = [label_list.index(label) for label in labels]
    return idxs

# Put to nan the labels that are not in MCS_6h_labels
label_6h_mask = label_mask.where(label_mask.isin(MCS_6h_labels))

## Precip Max over MCSs lifetimes

In [9]:
## pickle load Precip_max_over_lifetime if it exists 
filename = 'Precip_max_over_lifetime_MCS_6h.pkl' # used for import or saving the object
path = '/homedata/mcarenso/data/'  # my desired directory path

if os.path.isfile(os.path.join(path, filename)):
    #load it 
    with open(os.path.join(path, filename), 'rb') as file:
        Precip_max_over_lifetime = pickle.load(file)

else:
    ## We want to plot the maximim precip value found under MCS over its lifetime
    Precip_max_over_lifetime = []

    # I need fast acces to the precips so I will load them in memory
    Prec = ds["Prec"].load()

    for mcs in MCS_6h : 
        lifetime = []
        # get the label of the mcs 
        label = mcs.label
        # use label_mask to get the indexes of the label over t,y,x # this looks bad...
        mcs_idxs = np.where(label_6h_mask.compute() == label)
        # for each time steps of the mcs which is the first element of the tuple mcs_idxs retrieve the max precip value
        for t in np.unique(mcs_idxs[0]):
            y_idx,x_idx  = np.array(mcs_idxs[1][mcs_idxs[0]==t]), np.array(mcs_idxs[2][mcs_idxs[0]==t])
            max_precip = int(np.max(Prec[t,y_idx,x_idx]))
            lifetime.append(max_precip)
        
        Precip_max_over_lifetime.append(lifetime)
    #save it 
    with open(os.path.join(path, filename), 'wb') as file:
        pickle.dump(Precip_max_over_lifetime, file)
        

In [12]:
# get rid of empty list in Precip_max_over_lifetime
Precip_max_over_lifetime = [l for l in Precip_max_over_lifetime if l != []]
len(Precip_max_over_lifetime)

11